In [1]:
import subprocess as sp

from __init__ import *
import snmcseq_utils
from snmcseq_utils import cd
from snmcseq_utils import create_logger
from CEMBA_update_mysql import connect_sql
from natsort import natsorted

In [2]:
log = create_logger()
time_stamp = time.time()

In [5]:
# split allc tables of an ensemble to /scratch/tmp_allc_split
# /scratch/tmp_allc_split needs to be removed

ens = 'Ens2'

output_folder = '/scratch/tmp_allc_split_{}'.format(time_stamp)
try:
    os.makedirs(output_folder)
    logging.info('{} created!'.format(output_folder))
except:
    raise ValueError

logging.info("Splitting allc files to {}".format(output_folder))
for allc_file in sorted(glob.glob(os.path.join(PATH_ENSEMBLES, ens, 'allc_merged/allc_merged_*.tsv'))):
    # split allc files
    logging.info('Processing {}...'.format(os.path.basename(allc_file)))
    df = snmcseq_utils.read_allc_CEMBA(allc_file, pindex=False, compression='infer')
    for chrom, df_chrom in df.groupby('chr'):
        output = '{}/{}_{}.tsv'.format(output_folder, 
            os.path.basename(allc_file)[:-len('.tsv')], chrom) 
        df_chrom.to_csv(output, sep="\t", na_rep='NA', index=False, header=False)

03/07/2018 08:09:16 PM /scratch/tmp_allc_split_1520482155.2838366 created!
03/07/2018 08:09:16 PM Splitting allc files to /scratch/tmp_allc_split_1520482155.2838366
03/07/2018 08:09:16 PM Processing allc_merged_mCG_cluster_mCHmCG_lv_npc50_k30_10_Ens2.tsv...
03/07/2018 08:12:38 PM Processing allc_merged_mCG_cluster_mCHmCG_lv_npc50_k30_11_Ens2.tsv...
03/07/2018 08:16:05 PM Processing allc_merged_mCG_cluster_mCHmCG_lv_npc50_k30_12_Ens2.tsv...
03/07/2018 08:19:21 PM Processing allc_merged_mCG_cluster_mCHmCG_lv_npc50_k30_13_Ens2.tsv...
03/07/2018 08:22:32 PM Processing allc_merged_mCG_cluster_mCHmCG_lv_npc50_k30_14_Ens2.tsv...
03/07/2018 08:25:36 PM Processing allc_merged_mCG_cluster_mCHmCG_lv_npc50_k30_15_Ens2.tsv...
03/07/2018 08:28:41 PM Processing allc_merged_mCG_cluster_mCHmCG_lv_npc50_k30_16_Ens2.tsv...
03/07/2018 08:31:42 PM Processing allc_merged_mCG_cluster_mCHmCG_lv_npc50_k30_17_Ens2.tsv...
03/07/2018 08:34:26 PM Processing allc_merged_mCG_cluster_mCHmCG_lv_npc50_k30_18_Ens2.tsv..

In [6]:
# rename splitted allc for further processes (added clst removed merged)
allc_split_files = sorted(glob.glob(os.path.join(output_folder, 'allc_merged_*.tsv')))

for file in allc_split_files:
    src = file
    file = os.path.basename(file)
    file_l = file.split('_')
    file_l[-3] = 'clst' + file_l[-3]
    file = '_'.join(file_l)
    dst = os.path.join(output_folder, file)
    os.rename(src, dst)
    
allc_split_files = sorted(glob.glob(os.path.join(output_folder, 'allc_merged_*.tsv')))

In [7]:
# allc to mc
# mc files are in /scratch -> /scratch/tmp_mc_split_$time

# # ***
# logging.info("allc to mc...")
# output_folder = '/scratch/tmp_allc_split'
# allc_split_files = glob.glob(os.path.join(output_folder, 'allc_*.tsv'))
# # ***

logging.info("Allc to mc files...")
allc_split_files = sorted(glob.glob(os.path.join(output_folder, 'allc_merged_*.tsv')))
mc_split_files = ['mc_' + os.path.basename(file)[len('allc_'):] 
                  for file in allc_split_files]

# allc to mc
with cd(output_folder):
    cmd = '/cndd/fangming/lab_scripts/allc2mc.pl ./allc_merged_*.tsv'
    sp.call(cmd, shell=True)

# move all relevent mc files to a tmp folder
output_folder_mc = '/scratch/tmp_mc_{}'.format(time_stamp) 
try:
    os.makedirs(output_folder_mc)
    logging.info('{} created!'.format(output_folder_mc))
except:
    raise ValueError
    
for mc_split_file in mc_split_files:
    src = os.path.join('/scratch', mc_split_file)
    dst = os.path.join(output_folder_mc, mc_split_file)
    os.rename(src, dst)

03/07/2018 09:10:24 PM Allc to mc files...
03/07/2018 09:29:07 PM /scratch/tmp_mc_1520482155.2838366 created!


In [8]:
# mc to mysql 

# cd output_folder_mc
logging.info('Mc files to mySQL...')
with cd(output_folder_mc):
    cmd = '/cndd/fangming/lab_scripts/load_mysql_mc CEMBA_annoj > /cndd/fangming/CEMBA/snmcseq_dev/logs/annoj_1.log'
    sp.call(cmd, shell=True)

03/07/2018 09:29:07 PM Mc files to mySQL...


In [ ]:
# # clean up: allc_split, mc

# # ***
# output_folder = '/scratch/tmp_allc_split'
# # ***

# cmd = 'rm -r {}'.format(output_folder)
# sp.call(cmd, shell=True)
# cmd = 'rm -r {}'.format(output_folder_mc)
# sp.call(cmd, shell=True)
# logging.info("Tmp folders removed: {} {}".format(output_folder, output_folder_mc))


In [9]:
# dmr to bed

logging.info("DMR to bed files...")

cluster_type = 'cluster_mCHmCG_lv_npc50_k30'
annotation_type = 'annotation' + cluster_type[len('cluster'):]
engine = connect_sql('CEMBA')
output_folder_dmr = '/scratch/tmp_dmr_{}'.format(time_stamp)

# cluster annotation
sql = """SELECT {}, {} FROM {}""".format(cluster_type, annotation_type, ens)
df_info = pd.read_sql(sql, engine)
df_info = df_info.sort_values(cluster_type).drop_duplicates().set_index(cluster_type)

ens_path = os.path.join(PATH_ENSEMBLES, ens)
n_clusters = len(glob.glob(os.path.join(ens_path, 'allc_merged/allc_merged_mCG_{}_*_{}.tsv'.format(cluster_type, ens))))

# dmr results
input_f = os.path.join(ens_path, 'dmr/dmr_allc_merged_mCG_{}_rms_results_collapsed.tsv'.format(cluster_type))
df = pd.read_table(input_f, index_col=['#chr', 'start', 'end'], dtype={'#chr': object})

# df_hypo
df_hypo = df.loc[(df['number_of_dms']>=3) & (~df['hypomethylated_samples'].isnull()), 'hypomethylated_samples'].apply(
        lambda x: x.split(','))
# df_hyper = df.loc[(df['number_of_dms']>=3) & (~df['hypermethylated_samples'].isnull()), 'hypermethylated_samples'].apply(lambda x: x.split(','))

try:
    os.makedirs(output_folder_dmr)
    logging.info('Created directory: {}'.format(output_folder_dmr))
except:
    raise ValueError
    
for i in range(n_clusters):
    df_hypo_cluster = df_hypo.loc[
        df_hypo.apply(lambda x: ('merged_mCG_{}_{}_{}'.format(cluster_type, i+1, ens) in x))]
    output = os.path.join(output_folder_dmr, 'dmr_hypo_allc_merged_mCG_cluster_mCHmCG_lv_npc50_k30_clst{}.bed'.format(i+1))
    df_hypo_cluster.to_csv(output, sep='\t', header=False, index=True, na_rep='NA')
    logging.info("Saved to {}".format(output))
    logging.info(df_hypo_cluster.shape)

03/07/2018 09:38:06 PM DMR to bed files...
03/07/2018 09:38:09 PM Created directory: /scratch/tmp_dmr_1520482155.2838366
03/07/2018 09:38:10 PM Saved to /scratch/tmp_dmr_1520482155.2838366/dmr_hypo_allc_merged_mCG_cluster_mCHmCG_lv_npc50_k30_clst1.bed
03/07/2018 09:38:10 PM (29057,)
03/07/2018 09:38:11 PM Saved to /scratch/tmp_dmr_1520482155.2838366/dmr_hypo_allc_merged_mCG_cluster_mCHmCG_lv_npc50_k30_clst2.bed
03/07/2018 09:38:11 PM (30873,)
03/07/2018 09:38:12 PM Saved to /scratch/tmp_dmr_1520482155.2838366/dmr_hypo_allc_merged_mCG_cluster_mCHmCG_lv_npc50_k30_clst3.bed
03/07/2018 09:38:12 PM (25359,)
03/07/2018 09:38:13 PM Saved to /scratch/tmp_dmr_1520482155.2838366/dmr_hypo_allc_merged_mCG_cluster_mCHmCG_lv_npc50_k30_clst4.bed
03/07/2018 09:38:13 PM (13053,)
03/07/2018 09:38:14 PM Saved to /scratch/tmp_dmr_1520482155.2838366/dmr_hypo_allc_merged_mCG_cluster_mCHmCG_lv_npc50_k30_clst5.bed
03/07/2018 09:38:14 PM (4382,)
03/07/2018 09:38:14 PM Saved to /scratch/tmp_dmr_1520482155.28383

In [10]:
# rename bed
with cd(output_folder_dmr):
    for file in glob.glob('*'):
        src = file 
        dst = '{}_{}.bed'.format(src[:-len('.bed')], ens).replace('allc_merged_', '') 
        os.rename(src, dst)

In [11]:
# bed to mysql

logging.info("Uploading bed files (DMR) to mySQL databases...")
bed_files = sorted(glob.glob(os.path.join(output_folder_dmr, '*')))
with cd(output_folder_dmr):
    for bed_file in bed_files:
        # fn, db, table_prefix
        cmd = '/cndd/fangming/lab_scripts/bed_load_mysql.sh {} {} {}'.format(os.path.basename(bed_file), 
                                                                             'CEMBA_annoj', 
                                                                             os.path.basename(bed_file)[:-len('.bed')])
#         print(cmd)
        sp.call(cmd, shell=True)


03/07/2018 09:38:23 PM Uploading bed files (DMR) to mySQL databases...


In [ ]:
# # clean up: bed 
# cmd = 'rm -r {}'.format(output_folder_dmr)
# sp.call(cmd, shell=True)
# logging.info("Tmp folders removed: {}".format(output_folder_dmr))

In [1]:
# create php file
path = '/var/www/html/annoj_private/CEMBA'

In [4]:
TEMPLATE_MC_PHP = (
"""<?php
$append_assembly = true;
$table = 'CEMBA_annoj.{}_';
$title = '{}';
$info = '{}';
$link = mysql_connect("banjo.ucsd.edu", "cndd_annoj", "jonna_ddnc") or die("failed");
require_once '../includes/common_wgta.php';
?>
"""
)

TEMPLATE_BED_PHP = (
"""<?php
$append_assembly = true;
$table = 'CEMBA_annoj.{}_';
$title = '{}';
$info = '{}';
$link = mysql_connect("banjo.ucsd.edu", "cndd_annoj", "jonna_ddnc") or die("failed");
require_once '../includes/common_masks.php';
?>
"""
)

TEMPLATE_DMR_PHP = TEMPLATE_BED_PHP 

TEMPLATE_ATAC_PHP = TEMPLATE_BED_PHP 

def gen_php_template(track_name, template_string):
    """
    """
    return template_string.format(track_name, track_name, track_name)

In [3]:
# # check what's in mysql
# track_names = ['dmr_hypo_allc_merged_mCG_cluster_mCHmCG_lv_npc50_k30_clst{}'.format(i+1) for i in range(17)] 
# for track_name in track_names:
#     with open(os.path.join(path, 'browser/fetchers/dmrs/{}.php'.format(track_name)), 'w') as f_php:
#         f_php.write(gen_php_template(track_name, TEMPLATE_DMR_PHP))

In [16]:
# create php files
# create index files

In [17]:
# mc_split_file
output_folder_php = '/cndd/fangming/CEMBA/snmcseq_dev/browsers/php_mc_{}'.format(ens)
if not os.path.isdir(output_folder_php):
    os.makedirs(output_folder_php)
    logging.info('Created directory: {}'.format(output_folder_php))
    
track_names_mc = np.unique(['_'.join(os.path.basename(file).split('_')[:-1]) for file in mc_split_files])
for track_name in track_names_mc:
    with open(os.path.join(output_folder_php, '{}.php'.format(track_name)), 'w') as f_php:
        f_php.write(gen_php_template(track_name, TEMPLATE_MC_PHP))

# dmr files
output_folder_php = '/cndd/fangming/CEMBA/snmcseq_dev/browsers/php_dmrs_{}'.format(ens)
if not os.path.isdir(output_folder_php):
    os.makedirs(output_folder_php)
    logging.info('Created directory: {}'.format(output_folder_php))
track_names_dmr = [os.path.basename(file)[:-len('.bed')] for file in bed_files]
for track_name in track_names_dmr:
    with open(os.path.join(output_folder_php, '{}.php'.format(track_name)), 'w') as f_php:
        f_php.write(gen_php_template(track_name, TEMPLATE_DMR_PHP))


In [37]:
# index_file
TEMPLATE_INDEX_MC = (
"""
    {{
      id   : '{}',
      name : '{}',
      type : 'MethTrack',
      path : 'DNA Methylation',
      data : './browser/fetchers/mc/{}.php',
      iconCls : 'salk_meth',
      height : 30,
      class : 'CG -CHH -CHG',
      showControls : true
    }},
"""
)

TEMPLATE_INDEX_MC_SINGLE = (
"""
    {{
      id   : '{}',
      name : '{}',
      type : 'MethTrack',
      path : 'DNA Methylation',
      data : './browser/fetchers/mc_single/{}.php',
      iconCls : 'salk_meth',
      height : 30,
      class : 'CGdmr CG COV', 
      single : true,
      showControls : true
    }},
"""
)

TEMPLATE_INDEX_DMR = (
"""
    {{
      id : '{}',
      name : '{}',
      type : 'ReadsTrack',
      path : 'DMR',
      data : './browser/fetchers/dmrs/{}.php',
      iconCls : 'salk_dmr',
      height : 5,
      scale : 100,
      color : {{read : '000099'}},
      showControls : 0,
      single : true,
    }},
"""
)
TEMPLATE_INDEX_ATAC = (
"""
      {{
        id : '{}',
        name : '{}',
        type : 'PairedEndTrack',
        path : 'ATAC-Seq',
        data : './browser/fetchers/atac/{}.php',
        iconCls : 'silk_bricks',
        height : 25,
        scale : 5,
        single : true,
        color : {{read : '#0770af'}},
      }},
"""
)

TEMPLATE_INDEX_RNA = (
"""
      {{
        id : '{}',
        name : '{}',
        type : 'PairedEndTrack',
        path : 'RNA-Seq',
        data : './browser/fetchers/mrna/{}.php',
        iconCls : 'silk_bricks',
        height : 25,
        scale : 1,
        single : true,
        color : {{read : '#af0770'}},
      }},
"""
)

def gen_annoj_config(track_name, track_vis_name, template_string):
    """
    """
    return template_string.format(track_name, track_vis_name, track_name)

def gen_annoj_configs(track_names, track_vis_names, template_string):
    """
    """
    string = ''
    for track_name, track_vis_name in zip(track_names, track_vis_names):
        string += template_string.format(track_name, track_vis_name, track_name)
    return string

def gen_active_configs(lst):
    string = ''
    for item in lst:
        string +="\t'{}',\n".format(item)
    return string

In [45]:
TEMPLATE_INDEX_HTML=(
"""<html>
<head>
  <meta http-equiv='Content-Type' content='text/html; charset=iso-8859-1'>
  <title>{}</title>


  <link type='text/css' rel='stylesheet' href='aj2/aj2_css_dev/ext-all.css' />
  <script type='text/javascript' src='aj2/js/ext-base-3.2.js'></script>
  <script type='text/javascript' src='aj2/js/ext-all-3.2.js'></script>

  <link type='text/css' rel='stylesheet' href='aj2/aj2_css_dev/viewport.css' />
  <link type='text/css' rel='stylesheet' href='aj2/aj2_css_dev/plugins.css' />
  <link type='text/css' rel='stylesheet' href='aj2/aj2_css_dev/salk.css' />
  <script type='text/javascript' src='aj2/js/excanvas.js'></script>
  <script type='text/javascript' src='aj2/js/aj-cndd1-dev.js'></script>

  <!-- Favicon -->
  <link rel="icon" href="/var/www/html/annoj/browser/aj.ico" type="image/x-icon">
  <link rel="shortcut icon" href="/var/www/html/annoj/browser/aj.ico" type="image/x-icon">


  <!-- Config -->
  <script type='text/javascript'>

  AnnoJ.config = {{

    info : {{
      title  : '{}',
      genome  : 'mm10',
      contact  : '',
      email : '',
      institution : ''
    }},
    tracks : [
      //Models
      {{
        id   : 'gene_model_mm10',
        name : 'Gene Models (mm10)',
        type : 'ModelsTrack',
        path : 'Annotation models',
        data : './browser/fetchers/models/genes_mm10.php',
        height : 100,
        showControls : true
      }},


      // DNA methylation
      {}
      // DMRs
      {}
      // ATAC 
      {}
      // RNA 
      {}
      // repenh 
      {}


    ],

    active : [// *** Gene models
      'gene_model_mm10',
      {}

    ],
    genome    : './browser/fetchers/mus_musculus.php',
    bookmarks : './browser/fetchers/mus_musculus.php',
    stylesheets : [
      {{
        id   : 'css1',
        name : 'Plugins CSS',
        href : 'css/plugins.css',
        active : true
      }},{{
        id   : 'css2',
        name : 'SALK CSS',
        href : 'css/salk.css',
        active : true
      }}
    ],
    location : {{
      assembly : '1',
      position : '132487763',
      bases    : 300,
      pixels   : 1
    }},
    admin : {{
      name  : 'Eran Mukamel',
      email : 'emukamel@ucsd.edu',
      notes : 'University of California, San Diego'
    }},
  }};



  </script>



</head>

<body>

  <noscript>
    <table id='noscript'><tr>
      <td><img src='hs/img/Anno-J.jpg' /></td>
      <td>
        <p>Anno-J cannot run because your browser is currently configured to block Javascript.</p>
        <p>To use the application please access your browser settings or preferences, turn Javascript support back on, and then refresh this page.</p>
        <p>Thankyou, and enjoy the application!<br /></p>
      </td>
    </tr></table>
  </noscript>
  
  <!-- Enable URL queries -->
  <script type='text/javascript'> var queryPost; </script>
  <script type='text/javascript' src='./browser/js/urlinit.js'></script>
  
  <!-- Google Analytics -->
  <script type="text/javascript">
  var gaJsHost = (("https:" == document.location.protocol) ? "https://ssl." : "http://www.");
  document.write(unescape("%3Cscript src='" + gaJsHost + "google-analytics.com/ga.js' type='text/javascript'%3E%3C/script%3E"));
  </script>

  <script type="text/javascript">
  var pageTracker = _gat._getTracker("UA-4150298-1");
  pageTracker._initData();
  pageTracker._trackPageview();
  </script>
  <script type="text/javascript">
  var gaJsHost = (("https:" == document.location.protocol) ? "https://ssl." : "http://www.");
  document.write(unescape("%3Cscript src='" + gaJsHost + "google-analytics.com/ga.js' type='text/javascript'%3E%3C/script%3E"));
  </script>
  <script type="text/javascript">
  try {{
    var pageTracker = _gat._getTracker("UA-4150298-1");
    pageTracker._trackPageview();
  }} catch(err) {{}}</script>
</body>

</html>
""" # .format(title, title, mc, dmr, atac, rna, active)
)

def gen_index_html(template_string, title, mc, dmr, atac, rna, repenh, active):
    return (template_string.format(title, title, mc, dmr, atac, rna, repenh, active))


In [39]:
# annotation
f = '/cndd/fangming/CEMBA/annoj_browser/multimodal_v1/annot.tsv'
df_annot = pd.read_table(f)
df_annot['cluster'] = [item[len('cluster_'):] for item in df_annot['cluster_ID']] 
df_annot = df_annot.set_index('cluster') 
df_annot.head()

def get_annot(df_annot, cluster):
    return df_annot.loc[cluster, 'cluster_annotation']

In [51]:
table_prefixes_repenh

['multimodal_v1_DL_reptile_enhancers',
 'multimodal_v1_Glia_reptile_enhancers',
 'multimodal_v1_L2_reptile_enhancers',
 'multimodal_v1_L4_reptile_enhancers',
 'multimodal_v1_L5a_reptile_enhancers',
 'multimodal_v1_L5b_reptile_enhancers',
 'multimodal_v1_L6_reptile_enhancers',
 'multimodal_v1_Ndnf_reptile_enhancers',
 'multimodal_v1_Pv_reptile_enhancers',
 'multimodal_v1_Sst_reptile_enhancers',
 'multimodal_v1_Vip_reptile_enhancers',
 'multimodal_v1_mESC_reptile_enhancers']

In [58]:
title = 'CEMBA multimodal (v1) browser ({})'.format(ens)

table_prefixes_mc = natsorted(table_prefixes_mc) 
table_prefixes_dmr = natsorted(table_prefixes_dmr) 
table_prefixes_atac = natsorted(table_prefixes_atac) 
table_prefixes_rna = natsorted(table_prefixes_rna) 
table_prefixes_repenh = natsorted(table_prefixes_repenh) 


cluster_ids_mc = [item[len('mc_multimodal_v1_C'):] for item in table_prefixes_mc]
cluster_ids_dmr = [item[len('multimodal_v1_C'):-len('_cgdmr')] for item in table_prefixes_dmr]
cluster_ids_atac = [item[len('multimodal_v1_C'):-len('_atac')] for item in table_prefixes_atac]
cluster_ids_rna = [item[len('multimodal_v1_C'):-len('_rna')] for item in table_prefixes_rna]
# print(cluster_ids_mc, 
#       cluster_ids_dmr, 
#       cluster_ids_atac, 
#       cluster_ids_rna, 
#      )

table_vis_prefixes_mc = ['multimodal_v1_C{}_{}_mCG'
        .format(item, df_annot.loc[item, 'cluster_annotation']) for item in cluster_ids_mc]
table_vis_prefixes_dmr = ['multimodal_v1_C{}_{}_DMR'
        .format(item, df_annot.loc[item, 'cluster_annotation']) for item in cluster_ids_dmr]
table_vis_prefixes_atac = ['multimodal_C{}_{}_ATAC'
        .format(item, df_annot.loc[item, 'cluster_annotation']) for item in cluster_ids_atac]
table_vis_prefixes_rna = ['multimodal_C{}_{}_snRNA'
        .format(item, df_annot.loc[item, 'cluster_annotation']) for item in cluster_ids_rna]

table_vis_prefixes_repenh = ['multimodal_v1_{}_reptile_enhancer'
        .format(item.split('_')[2]) for item in table_prefixes_repenh]

# print(table_vis_prefixes_mc, 
#       table_vis_prefixes_dmr, 
#       table_vis_prefixes_atac, 
#       table_vis_prefixes_rna, 
#       table_vis_prefixes_repenh)

mc_configs = gen_annoj_configs(table_prefixes_mc, table_vis_prefixes_mc, TEMPLATE_INDEX_MC_SINGLE) 
dmr_configs = gen_annoj_configs(table_prefixes_dmr, table_vis_prefixes_dmr, TEMPLATE_INDEX_DMR) 
rna_configs = gen_annoj_configs(table_prefixes_rna, table_vis_prefixes_rna, TEMPLATE_INDEX_RNA) 
atac_configs = gen_annoj_configs(table_prefixes_atac, table_vis_prefixes_atac, TEMPLATE_INDEX_ATAC) 
repenh_configs = gen_annoj_configs(table_prefixes_repenh, table_vis_prefixes_repenh, TEMPLATE_INDEX_DMR) 

# active order (natsorted mc track and dmr tracks)
active_orders = np.concatenate([table_prefixes_repenh,
                          table_prefixes_mc[:5], 
#                           table_prefixes_dmr[:5],
                          table_prefixes_atac[:5], 
                          table_prefixes_rna[:5]])
active_configs = gen_active_configs(active_orders)
# print(active_configs)

index_html = gen_index_html(TEMPLATE_INDEX_HTML, title, 
                            mc_configs, dmr_configs, 
                            atac_configs, rna_configs, 
                            repenh_configs, 
                            active_configs)
print(index_html)

<html>
<head>
  <meta http-equiv='Content-Type' content='text/html; charset=iso-8859-1'>
  <title>CEMBA multimodal (v1) browser (Ens10-v1)</title>


  <link type='text/css' rel='stylesheet' href='aj2/aj2_css_dev/ext-all.css' />
  <script type='text/javascript' src='aj2/js/ext-base-3.2.js'></script>
  <script type='text/javascript' src='aj2/js/ext-all-3.2.js'></script>

  <link type='text/css' rel='stylesheet' href='aj2/aj2_css_dev/viewport.css' />
  <link type='text/css' rel='stylesheet' href='aj2/aj2_css_dev/plugins.css' />
  <link type='text/css' rel='stylesheet' href='aj2/aj2_css_dev/salk.css' />
  <script type='text/javascript' src='aj2/js/excanvas.js'></script>
  <script type='text/javascript' src='aj2/js/aj-cndd1-dev.js'></script>

  <!-- Favicon -->
  <link rel="icon" href="/var/www/html/annoj/browser/aj.ico" type="image/x-icon">
  <link rel="shortcut icon" href="/var/www/html/annoj/browser/aj.ico" type="image/x-icon">


  <!-- Config -->
  <script type='text/javascript'>

  Ann

In [59]:
output_html = '/cndd/fangming/CEMBA/annoj_browser/multimodal_v1/index_{}.html'.format(ens)
with open(output_html, 'w') as f:
    f.write(index_html)


In [23]:
table_prefixes_repenh = natsorted(table_prefixes_repenh) 
table_prefixes_dmr = natsorted(table_prefixes_dmr) 

table_vis_prefixes_repenh = ['multimodal_v1_{}_reptile_enhancer'
        .format(item[len('multimodal_v1_'):-len('_repenh')]) for item in table_prefixes_repenh]
table_vis_prefixes_dmr = ['multimodal_v1_C{}_DMR'
        .format(item[len('multimodal_v1_C'):-len('_cgdmr')]) for item in table_prefixes_dmr]

repenh_configs = gen_annoj_configs(table_prefixes_repenh, table_vis_prefixes_repenh, TEMPLATE_INDEX_DMR) 
dmr_configs = gen_annoj_configs(table_prefixes_dmr, table_vis_prefixes_dmr, TEMPLATE_INDEX_DMR) 

print(repenh_configs)

active_orders = natsorted(table_prefixes_repenh) 
print(gen_active_configs(active_orders))


    {
      id : 'multimodal_v1_DL_reptile_enhancers',
      name : 'multimodal_v1_DL_reptile_en_reptile_enhancer',
      type : 'ReadsTrack',
      path : 'DMR',
      data : './browser/fetchers/dmrs/multimodal_v1_DL_reptile_enhancers.php',
      iconCls : 'salk_dmr',
      height : 5,
      scale : 100,
      color : {read : '000099'},
      showControls : 0,
      single : true,
    },

    {
      id : 'multimodal_v1_Glia_reptile_enhancers',
      name : 'multimodal_v1_Glia_reptile_en_reptile_enhancer',
      type : 'ReadsTrack',
      path : 'DMR',
      data : './browser/fetchers/dmrs/multimodal_v1_Glia_reptile_enhancers.php',
      iconCls : 'salk_dmr',
      height : 5,
      scale : 100,
      color : {read : '000099'},
      showControls : 0,
      single : true,
    },

    {
      id : 'multimodal_v1_L2_reptile_enhancers',
      name : 'multimodal_v1_L2_reptile_en_reptile_enhancer',
      type : 'ReadsTrack',
      path : 'DMR',
      data : './browser/fetchers/dmrs/multim

In [20]:
table_prefixes_dmr = natsorted(table_prefixes_dmr) 
table_vis_prefixes_dmr = ['multimodal_v1_C{}_DMR'
        .format(item[len('multimodal_v1_C'):-len('_cgdmr')]) for item in table_prefixes_dmr]
dmr_configs = gen_annoj_configs(table_prefixes_dmr, table_vis_prefixes_dmr, TEMPLATE_INDEX_DMR) 

print(dmr_configs)
active_orders = natsorted(table_prefixes_dmr) 
# active_orders


    {
      id : 'multimodal_v1_C1_cgdmr',
      name : 'multimodal_v1_C1_DMR',
      type : 'ReadsTrack',
      path : 'DMR',
      data : './browser/fetchers/dmrs/multimodal_v1_C1_cgdmr.php',
      iconCls : 'salk_dmr',
      height : 5,
      scale : 100,
      color : {read : '000099'},
      showControls : 0,
      single : true,
    },

    {
      id : 'multimodal_v1_C2_cgdmr',
      name : 'multimodal_v1_C2_DMR',
      type : 'ReadsTrack',
      path : 'DMR',
      data : './browser/fetchers/dmrs/multimodal_v1_C2_cgdmr.php',
      iconCls : 'salk_dmr',
      height : 5,
      scale : 100,
      color : {read : '000099'},
      showControls : 0,
      single : true,
    },

    {
      id : 'multimodal_v1_C3_cgdmr',
      name : 'multimodal_v1_C3_DMR',
      type : 'ReadsTrack',
      path : 'DMR',
      data : './browser/fetchers/dmrs/multimodal_v1_C3_cgdmr.php',
      iconCls : 'salk_dmr',
      height : 5,
      scale : 100,
      color : {read : '000099'},
      showControl

In [76]:
track_vis_names_mc = [file.replace('merged_', '').replace('mc_', '') for file in track_names_mc]
track_vis_names_dmr = track_names_dmr 

title = 'CEMBA browser ({})'.format(ens)
mc_configs = gen_annoj_configs(track_names_mc, track_vis_names_mc, TEMPLATE_INDEX_MC) 
dmr_configs = gen_annoj_configs(track_names_dmr, track_vis_names_dmr, TEMPLATE_INDEX_DMR) 

# active order (natsorted mc track and dmr tracks)
from natsort import natsorted
active_orders = np.vstack((np.asarray(natsorted(track_names_mc)), 
                           np.asarray(natsorted(track_names_dmr)))).flatten('F')
active_configs = gen_active_configs(active_orders) 

index_html = gen_index_html(TEMPLATE_INDEX_HTML, title, mc_configs, dmr_configs, active_configs)


NameError: name 'track_names_mc' is not defined

In [28]:
# dmr php
ens = 'Ens10-v1'
output_folder_php = '/cndd/fangming/CEMBA/annoj_browser/multimodal_v1/php_dmr_{}'.format(ens)
with snmcseq_utils.cd('/cndd/fangming/CEMBA/annoj_browser/multimodal_v1/dmr_tracks'):
    files = glob.glob('*.bed')
    clsts = [file.replace('.', '_').split('_')[2][len('clst'):] for file in files]
    
    table_prefixes_dmr = ['multimodal_v1_C{}_cgdmr'.format(i) for i in clsts]
    print(table_prefixes_dmr)
    
if not os.path.isdir(output_folder_php):
    os.makedirs(output_folder_php)
    logging.info('Created directory: {}'.format(output_folder_php))
    
for table_prefix in table_prefixes_dmr:
    with open(os.path.join(output_folder_php, '{}.php'.format(table_prefix)), 'w') as f_php:
        f_php.write(gen_php_template(table_prefix, TEMPLATE_BED_PHP))

# reptile enhancer php
ens = 'Ens10-v1'
output_folder_php = '/cndd/fangming/CEMBA/annoj_browser/multimodal_v1/php_reptile_enhancer_{}'.format(ens)
with snmcseq_utils.cd('/cndd/fangming/CEMBA/annoj_browser/multimodal_v1/reptile_enhancer_tracks'):
    files = glob.glob('*.bed')
    clsts = [file.replace('.', '_').split('_')[1] for file in files]
    
    table_prefixes_repenh = ['multimodal_v1_{}_reptile_enhancers'.format(i) for i in clsts]
    print(table_prefixes_repenh)
    
if not os.path.isdir(output_folder_php):
    os.makedirs(output_folder_php)
    logging.info('Created directory: {}'.format(output_folder_php))
    
for table_prefix in table_prefixes_repenh:
    with open(os.path.join(output_folder_php, '{}.php'.format(table_prefix)), 'w') as f_php:
        f_php.write(gen_php_template(table_prefix, TEMPLATE_BED_PHP))


# rna php

ens = 'Ens10-v1'
output_folder_php = '/cndd/fangming/CEMBA/annoj_browser/multimodal_v1/php_rna_{}'.format(ens)
with snmcseq_utils.cd('/cndd/fangming/CEMBA/annoj_browser/multimodal_v1/snrna_tracks'):
    files = glob.glob('*.bed')
    clsts = [file.replace('.', '_').split('_')[2] for file in files]
    
    table_prefixes_rna = ['multimodal_v1_C{}_rna'.format(i) for i in clsts]
    print(table_prefixes_rna)
    
if not os.path.isdir(output_folder_php):
    os.makedirs(output_folder_php)
    logging.info('Created directory: {}'.format(output_folder_php))
    
for table_prefix in table_prefixes_rna:
    with open(os.path.join(output_folder_php, '{}.php'.format(table_prefix)), 'w') as f_php:
        f_php.write(gen_php_template(table_prefix, TEMPLATE_BED_PHP))
        
# atac php

ens = 'Ens10-v1'
output_folder_php = '/cndd/fangming/CEMBA/annoj_browser/multimodal_v1/php_atac_{}'.format(ens)
with snmcseq_utils.cd('/cndd/fangming/CEMBA/annoj_browser/multimodal_v1/atac_tracks'):
    files = glob.glob('*.bed')
    clsts = [file.replace('.', '_').split('_')[1] for file in files]
    
    table_prefixes_atac = ['multimodal_v1_C{}_atac'.format(i) for i in clsts]
    print(table_prefixes_atac)
    
if not os.path.isdir(output_folder_php):
    os.makedirs(output_folder_php)
    logging.info('Created directory: {}'.format(output_folder_php))
    
for table_prefix in table_prefixes_atac:
    with open(os.path.join(output_folder_php, '{}.php'.format(table_prefix)), 'w') as f_php:
        f_php.write(gen_php_template(table_prefix, TEMPLATE_BED_PHP))

# mc php

ens = 'Ens10-v1'
output_folder_php = '/cndd/fangming/CEMBA/annoj_browser/multimodal_v1/php_mc_{}'.format(ens)
with snmcseq_utils.cd('/cndd/fangming/CEMBA/annoj_browser/multimodal_v1/mc_tracks'):
    files = glob.glob('allc_*.tsv')
    clsts = [file.replace('.', '_').split('_')[3].strip('clst') for file in files]
    
    table_prefixes_mc = ['mc_multimodal_v1_C{}'.format(i) for i in clsts]
    print(table_prefixes_mc)
    
if not os.path.isdir(output_folder_php):
    os.makedirs(output_folder_php)
    logging.info('Created directory: {}'.format(output_folder_php))
    
for table_prefix in table_prefixes_mc:
    with open(os.path.join(output_folder_php, '{}.php'.format(table_prefix)), 'w') as f_php:
        f_php.write(gen_php_template(table_prefix, TEMPLATE_MC_PHP))

['multimodal_v1_C33_cgdmr', 'multimodal_v1_C48_cgdmr', 'multimodal_v1_C34_cgdmr', 'multimodal_v1_C2_cgdmr', 'multimodal_v1_C5_cgdmr', 'multimodal_v1_C25_cgdmr', 'multimodal_v1_C22_cgdmr', 'multimodal_v1_C17_cgdmr', 'multimodal_v1_C10_cgdmr', 'multimodal_v1_C19_cgdmr', 'multimodal_v1_C40_cgdmr', 'multimodal_v1_C4_cgdmr', 'multimodal_v1_C3_cgdmr', 'multimodal_v1_C35_cgdmr', 'multimodal_v1_C32_cgdmr', 'multimodal_v1_C18_cgdmr', 'multimodal_v1_C11_cgdmr', 'multimodal_v1_C16_cgdmr', 'multimodal_v1_C23_cgdmr', 'multimodal_v1_C24_cgdmr', 'multimodal_v1_C15_cgdmr', 'multimodal_v1_C12_cgdmr', 'multimodal_v1_C27_cgdmr', 'multimodal_v1_C20_cgdmr', 'multimodal_v1_C29_cgdmr', 'multimodal_v1_C38_cgdmr', 'multimodal_v1_C7_cgdmr', 'multimodal_v1_C31_cgdmr', 'multimodal_v1_C9_cgdmr', 'multimodal_v1_C36_cgdmr', 'multimodal_v1_C28_cgdmr', 'multimodal_v1_C21_cgdmr', 'multimodal_v1_C26_cgdmr', 'multimodal_v1_C13_cgdmr', 'multimodal_v1_C14_cgdmr', 'multimodal_v1_C37_cgdmr', 'multimodal_v1_C30_cgdmr', 'multi

In [12]:
# bed to mysql # adding atac readend tracks

logging.info("Uploading bed files (ATAC readq) to mySQL databases...")
dirc = '/cndd/projects/Public_Datasets/CEMBA/snATACSeq/Datasets/CEMBA_3C_171206/uniq_chrs_readends'
table_prefixes = []
with cd(dirc):
    bed_files = sorted(glob.glob('CEMBA_*_merged.cluster.C*.bed'))
    for bed_file in bed_files:
        # fn, db, table_prefix
        table_prefix = os.path.splitext(bed_file)[0].replace('.', '_')
        table_prefixes.append(table_prefix)
        cmd = '/cndd/fangming/lab_scripts/bed_load_mysql.sh {} {} {}'.format(bed_file, 
                                                                             'CEMBA_annoj', 
                                                                             table_prefix)
#         print(cmd)
#         sp.call(cmd, shell=True)

04/06/2018 10:36:18 AM Uploading bed files (ATAC readq) to mySQL databases...


In [10]:
# bed to mysql # adding atac readend tracks

logging.info("Uploading bed files (ATAC readq) to mySQL databases...")
dirc = '/cndd/fangming/CEMBA/annoj_browser/multimodal_v1/snrna_tracks'
table_prefixes = []
with cd(dirc):
    bed_files = sorted(glob.glob('rna.*.bed'))
    for bed_file in bed_files:
        # fn, db, table_prefix
        table_prefix = 'multimodal_v1_C{}_rna'.format(os.path.splitext(bed_file)[0].replace('.', '_').split('_')[2])
        table_prefixes.append(table_prefix)
        cmd = '/cndd/fangming/lab_scripts/bed_load_mysql.sh {} {} {}'.format(bed_file, 
                                                                             'CEMBA_annoj', 
                                                                             table_prefix)
        print(cmd)
        sp.call(cmd, shell=True)

06/12/2018 11:43:06 PM Uploading bed files (ATAC readq) to mySQL databases...


/cndd/fangming/lab_scripts/bed_load_mysql.sh rna.cluster_1.chr.bed CEMBA_annoj multimodal_v1_C1_rna
/cndd/fangming/lab_scripts/bed_load_mysql.sh rna.cluster_10.chr.bed CEMBA_annoj multimodal_v1_C10_rna


KeyboardInterrupt: 